# Stabla odlučivanja za ciljane marketinške kampanje

## Napomena:
Da bi ovaj notebook ispravno radio, morate u isti folder gdje ste njega skinuli skinuti i fileove (u njima su podatci, mozete ih otvoriti i pogledati s Notepadom ili nekim drugim text editorom):
* algebra.csv
* algebra_s.csv
* to_classify.csv
* dt.png


## 1. Priprema podataka

Stabla odlučivanja su jedan od najjednostavnijih i najintuitivnijih modela strojnog učenja. U ovom notebooku ćemo vas upoznati sa njihovim funkcioniranjem na primjeru podataka s neke fiktivne web stranice za online kupovinu.

Koristit ćemo jezik Python 3 i obilnu količinu primjera i ilustracija. Također, naučit ćete korak po korak kako izgleda Python programski kod, i bit će nam drago da vam ovo ujedno bude poticaj za učenje Pythona koji je u današnjem poslovnom svijetu iznimno koristan.

Prvo ćemo učitati podatke. Da biste to napravili, kliknete na sljedeću ćeliju (gdje piše "**In[   ]**"), i stisnete *Shift+Enter(Return)*. Pričekajte dok piše "**In[\*]**" i dok ne vidite "**In[1]**", tada je izračun gotov. Kao rezultat biste trebali dobiti prvih pet redaka tablice.

In [ ]:
import pandas as pd
podatci = pd.read_csv("algebra.csv")
podatci.head()

**Zadatak 1:** Vidimo da ova kratka skripta (odnosno program, ili aplikacija) izlistava prvih pet redaka iz tablice "algebra.csv". U donjoj ćeliji pokušajte doraditi skriptu da izlistava prvih deset redaka:

In [ ]:
podatci.head(5)#modificirajte ovu liniju da dobijete 10 redaka

## 2. Ideja iza klasifikatora

Stablo odlučivanja je konceptualno najjednostavniji algoritam strojnog učenja, pa ćemo na njemu pokazati kako funkcionira strojno učenje. 

Algoritmi za strojno učenje se mogu ugrubo podijeliti na *klasifikacijske* algoritme i algoritme za *clustering*. Stablo odlučivanja je klasifikacijski algoritam.

Klasifikacija općenito funkcionira ovako. Jedan stupac iz tablice s podatcima proglasimo **ciljnom** varijablom. U našem slučaju je to *end_result* stupac, ali to može biti bilo koji drugi stupac koji nas zanima (odnosno za kojeg želimo saznati kako ga vrijednosti u ostalim stupcima *predviđaju*)

Dignimo prva tri stupca naše tablice (stisnite *Shift+Enter* na sljedećoj ćeliji):

In [ ]:
podatci.head(3)

Značenje stupca *end_result* je sljedeći:
* Vrijednost 0 znači "napuštena košarica"
* Vrijednost 1 znači "nedovršeno plaćanje"
* Vrijednost 2 znači "kartica odbijena prilikom plaćanja"
* Vrijednost 3 znači "Uspješno plaćanje"

Zamislimo za sada da su ova tri retka. Ako nam ciljna varijabla *end_result*, onda znači da mi želimo predvidjeti vrijednost *end_result* temeljem drugih stupaca. Na primjer, ako želimo predvidjeti tko je napustio košaricu, vidimo da vrijedi na ova tri retka "Ako **gender = m**, onda **end_result = 2**".

**gender = m** se naziva *prediktorom* za ciljnu varijablu **end_result** (odnosno za prepoznavanje kada je **end_result = 2**). Uočimo da je **gender = m** prediktor, dok se samo **gender** naziva *svojstvo* odnosno *feature* 

Prediktora može biti više, i na ovom primjeru vidimo da ih ima još:

* **gender = m**
* **age < 28** (iako je ovdje moglo biti i 24, 25, 26,...,32, ali smo odabrali 28 jer je na sredini)
* **item_id > 250** (iako opet, može biti bilo koja vrijednost koja ih razdvaja)
* Možete li naći jos neki? (hint: cijena)

Uočimo također da prediktori mogu biti složeni. Na primjer, dobro funkcionira i prediktor **prev_purchases_in_usd = 0 AND uses_whishlist = 1**. Ovaj prediktor isto upućuje na **end_result = 2**.

Kao mala digresija, kako bismo formulirali prediktor uz pomoć featurea **prev_purchases_in_usd = 0** i **uses_whishlist = 1** za **end_result = 2**?

Odgovor je **prev_purchases_in_usd > 0 OR uses_whishlist = 0** (uočite promjene, AND u OR, i vrijednosti).

No da se vratimo na stabla odlučivanja. Učitajmo sljedeći redak tablice:

In [ ]:
podatci.head(4)

Sada više nisu dobri prediktori **gender = m**, **age < 28**, **item_id > 0**, kao ni naš složeni prediktor. Prediktor temeljen na cijeni još uvijek radi. Učitajmo još jedan redak tablice:


In [ ]:
podatci.head(5)

U teoriji bismo mogli ručno konstruirati prediktore tako da hvatamo točne slučajeve. Na primjer, **price < 20 AND price > 95**, ili još "sigurnije" bismo uzeli **customer_id = 1001 OR customer_id = 1922 OR customer_id = 1004**. Ovime bismo išli u totalno krivom smjeru. 

Mi želimo stvoriti alat koji će nam predvidjeti za buduće kupce hoće li odustati ili ne, a metoda pobrojavanja po **customer_id** (znači fokus samo na ovaj stupac bez razmišljanja o drugim featurima) tko je napustio do sada nam u tome sigurno neće pomoći da bismo pridobili klijenta **customer_id = 5555**, ali ni za zadržati klijenta **customer_id = 0003**.


## 3. Klasifikacija i marketing

Ovdje je primjereno dati jednu važnu maksimu:

_**Akvizicija novih klijenata i retencija postojećih klijenata su dvije glavne zadaće marketinga**._ 

Naglasak može biti malo više na jednom ili na drugom, ali ovo je *esencija* marketinga. 

Mi želimo iz podataka "naučiti" kombinaciju svojstava koju dijele klijente koji su odustali od ostalih, i onda temeljem te kombinacije probati vidijeti u čemu nam je problem, ali i predvidjeti hoće li kupac 5555 imati isti problem, i time moći preventivno djelovati na njegovu retenciju. Retencija je u praksi najčešće otklanjanje poteskoća klijentima, jer je glavni razlog odlaska (na koji možemo djelovati), neki problem na našem sustavu. Naravno postoje i drugi razlozi, ali oni su (a) bitno teži za naći, i (b) bitno je teže na njih djelovati.

Drugi aspekt ovakve analize je vidjeti gdje ima mjesta za akviziciju. Akvizicija je u jednu ruku jednostavnija, a u jednu kompleksnija.

Jednostavnija je u tu ruku da je potrebno naći najbolji prediktor za **end_result = 3**, i analizirati rezultate. Uz malo sreće, među najboljim rezultatima ćemo imati vrijedne spoznaje, npr. **age > 25 AND age < 40 AND gender = f**, čime onda možemo raditi vrlo dobre ciljane kampanje.

Kompleksnija pak u tome da se akvizicija može gledati i na način da se radi *cross-selling*, odnosno da, ako smo naučili iz naših podataka da klijenti koji kupe bilo koju knjigu za preko 70 USD najčešće kupe i artikl 258, onda svakom tko stavi u košaricu knjigu u vrijednosti preko 70 USD želimo ponuditi i artikl 258 (možda sa popustom).


## 4. Stabla odlučivanja

Stabla odlučivanja biraju feature i kombiniraju ih tako da se dobije određeni redoslijed njihove primjene. Glavni trik je u tome da se mjeri koji featurei daju najbolju diskriminaciju, i oni se uzimaju prvi. Dobar primjer je predviđanje tko je (i tko bi) preživio na _Titanicu_ (posuđujemo sliku s Wikipedije)

<img src="dt.png">

Brojevi ispod elemenata slike označavaju vjerojatnost preživljavanja (decimalni brojevi oblika 0.34), i postotak sveukupnih slučajeva koji se nalaze u toj kategoriji (koji, radi zaokruživanja su u zbroju 101%, ali ovo nije važno sad). Vjerojatnost preživljavanja iznad 0.5 se smatra kategorijom "survived", a ispod 0.5 se smatra kategorijom "died" (ova tehnika gdje se brojevna skala podijeli na dvije kategorije prije i nakon neke granice se naziva "thresholding").

Znači, ako je osoba ženskog spola, njena vjerojatnost za preživljavanje je 0.73 (73%). Ako je muškog spola i stariji od 9.5 godina, onda je vjerojatno umro (vjerojatnost za preživljavanje 0.17, odnosno 17%). Najbolju šansu za preživljavanje su imale muške osobe mlađe od 9.5 godina koje imaju manje od 2.5 braće i sestara (0.89, odnosno 89% šanse, premda ih je bilo jako malo, odnosno 2% od sveukupne populacije). Ovih 2% znači da je bilo dosta malo uzoraka s ovim prediktorom (**sex = m AND age < 9.5 AND sibsp < 2.5**), što znači da možda ova klasifikacija nije najpouzdanija, odnosno da je možda preovisna o slučajnostima.

Klasifikacija na temelju spola daje 0.73 šanse za preživljavanje, ali ovo ima sposobnost na jednostavan način (uz samo feature **sex**) uhvatiti 36% sveukupnih podataka. U izvjesnom smislu, to je najjači feature i najstabilniji (što prati i zdravorazumsku intuiciju). 

Ovo je temelj funkcioniranja stabala odlučivanja: stablo odlučivanja pokušava naći redoslijed tako da prvo razgraničava po najjačem featureu, a zatim po sve slabijim. 

To **ne** znači da je najjači feature onaj koji daje najveće šanse za preživljavanje (odnosno najveću razliku oko *thresholdinga*), jer to nije više bitno, ono što je bitno je li iznad 0.5, pa dobije oznaku "PREŽIVLJAVA" ili ispod, pa dobije oznaku "UMIRE". 

Najjači feature je onaj koji najbolje dijeli uzorke, znači u našem slučaju 36% vs 64%. Detalji kako stablo točno računa kojim će redom ići su kompleksni (i upravo zato i koristimo algoritam a ne radimo ovo ručno), ali ovo opisano je intuicija iza toga.

Sada smo spremni dignuti stablo odlučivanja nad našim podatcima. Sljedećih nekoliko ćelija treba izvršiti sa Shift+Enter, a ispod njih dajemo objašnjenja.

In [ ]:
import pandas as pd
from sklearn import tree

Kratak komentar:

* Prvi redak služi za učitavanje dodatnog modula "Pandas" koji služi kao tablica (slično Excelu),
* Drugi redak učitava dodatni Python modul (sklearn) za njegov podmodul (tree) za stabla odlučivanja,

Sljedeći korak je da učitamo podatke (Shift+Enter na donju ćeliju):

In [ ]:
podatci_2 = pd.read_csv("algebra_s.csv")

**Napomena:** ovo nije ista tablica kao gornja, nego je reduciran broj stupaca (retci su isti). Izvršite donju ćeliju da vidite prvih par redaka:

In [ ]:
podatci_2.head()

Stupac **customer_id** nam više ne treba, pa ćemo ga se riješiti:

In [ ]:
podatci_3 = podatci_2.drop('customer_id',axis=1)
podatci_3.head()#ovaj redak je da provjerimo jesmo li dobro napravili

Sljedeći je korak podijeliti podatke u dvije tablice, jednu koja ima feature, a druga ima **end_result** u sebi (ovo radimo zato što SKlearn tako želi primati podatke):

In [ ]:
svojstva = podatci_3[["age","price"]]
svojstva.head()

In [ ]:
rezultati = podatci_3[["end_result"]]
rezultati.head()

Prisjetimo se da "head" znači da se ispiše prvih nekoliko rezultata (default=5, ako se želi neka druga vrijednost, treba se napisati u zagrade), a ako želimo vidjeti cijelu tablicu samo napisemo "rezultati" umjesto "rezultati.head()".

In [ ]:
stablo = tree.DecisionTreeClassifier()
Breza = stablo.fit(svojstva, rezultati)

Prvi redak u ćeliji iznad stvara novo prazno stablo iz SKlearn modula, a drugi redak ga utrenira nad podatcima iz CSV tablice koju smo istraživali. Sad je stablo utrenirano i živi u sustavu pod imenom "Breza" i spremno je za klasificiranje novih podataka.

Zamislimo da sada dobijemo tablicu klijenata koji su upravo na stranici i kupuju, i to je tablica "to_classify.csv". Učitajmo tu tablicu (Shift+Enter na donju ćeliju)

In [ ]:
klijenti_na_stranici = pd.read_csv("to_classify.csv") #ovo ispisuje tablicu
klijenti_na_stranici

Sada upogonimo Brezu nad ovim  podatcima:

In [ ]:
pred = Breza.predict(klijenti_na_stranici)
pred

Na ovaj je način Breza ispisala što će svaki od klijenata najvjerojatnije napraviti. Prisjetimo se:

* Vrijednost 0 znači "napuštena košarica"
* Vrijednost 1 znači "nedovršeno plaćanje"
* Vrijednost 2 znači "kartica odbijena prilikom plaćanja"
* Vrijednost 3 znači "Uspješno plaćanje"

Možemo ove podatke dodati u tablicu u novi stupac **expected_action**:

In [ ]:
klijenti_na_stranici['expected_action'] = pred
klijenti_na_stranici

## 5. Zadatci

Ovaj donji dio je predviđen za riješavanje zadataka vezanih uz ovaj notebook. Prema potrebi možete dodati ćelije ispod predviđene ćelije u nastavku. Naša je preporuka da svoj program napišete u Notepadu, ili još bolje u Notepad++, te ga zatim kopirate (ovo je napredna varijanta), ili da ga pišete liniju po liniju u ćeliju dolje, i nakon svake linije si isprintate rezultat da vidite jeste li dobili ono što ste htjeli dobiti. Naravno, možete i kombinirati ova dva pristupa.